November 2020

This notebook uses the NLCD Tree Canopy Cover Data to mask images of everything but tree cover.  You could probably adjust it for the more familiar Land Cover data, but you're on your own for that one.

The code is boiled down into a single function.  You just call it at the bottom of the notebook.

# Getting USFS NLCD Tree Canopy data

This is a big file, but I use it a lot. I save it in a central location so I can pull it for various projects. Just something to be aware of before you start.

Go to https://www.mrlc.gov/data?f%5B0%5D=category%3Aland%20cover&f%5B1%5D=region%3Aconus
(also the same place you'd get the Land Cover data)

Click the "Tree Canopy" button at the top, then scroll down to actually see the data.

Download the "NLCD 2016 USFS Tree Canopy Cover (CONUS)" (or newer if its available). Then go get a cup of joe because it'll take a while.  

Unzip it and save it to whatever folder you choose.

In [1]:
import os
from osgeo import gdal, gdal_array
import numpy as np
import numpy.ma as ma
import rasterio

In [3]:
### This is the only block you have to change ###

# This is where I save my image files and where the masked file will be saved.
os.chdir('filepath to images')

image_fps = ['image1.tif',
            'image2.tif']

nlcd_fp = '.../nlcd_2016_treecanopy_2019_08_31/nlcd_2016_treecanopy_2019_08_31.img'

sensor = 'Sentinel-2'  # You'll see your options below for sensors that can be used.
                       # Really all it does is automate the resampling in the next 
                       # block. Add to it as you'd like.

In [4]:
def nlcd_masking(image_fps, nlcd_fp, sensor):
    if sensor == 'Dove':
        res = 3
    elif sensor == 'Sentinel-2':
        res = 10
    elif sensor == 'Landsat' or sensor == 'HLS':
        res = 30   
        
    # Reproject mask (as needed) and align spatial resolution. Crop mask to image bounds.
    for image in image_fps[:1]:
        raster = rasterio.open(image)
        nlcd = rasterio.open(nlcd_fp)
        img_crs = raster.crs
        profile = nlcd.profile
        nlcd_crs = profile['crs']
        limits = [raster.bounds[0], raster.bounds[1], raster.bounds[2], raster.bounds[3]]
        # Generate gdal warp options 
        warp_options = gdal.WarpOptions(format='GTiff', 
                                        outputBounds=limits,
                                        xRes=res,                  
                                        yRes=res,
                                        srcSRS=nlcd_crs,
                                        dstSRS=img_crs,
                                        resampleAlg='near',
                                        multithread=True)

        nlcd_new = 'NLCD_Cropped_Reprojected_Resampled.tif' 

        create_nlcd_new = gdal.Warp(nlcd_new, nlcd_fp, options=warp_options)
        create_nlcd_new = None

    #Create mask
    nlcd_file = gdal.Open(nlcd_new)
    nlcd_data = nlcd_file.GetRasterBand(1)
    nlcd_array = gdal_array.BandReadAsArray(nlcd_data)

    # Select capopy pixels with greater than 50% canopy cover
    nlcd_mask = (nlcd_array < 50)

    # Pre-allocate an array 
    arr = np.zeros((np.shape(nlcd_mask)[0], np.shape(nlcd_mask)[1]),  dtype=np.uint16)  # pre-allocate   

    for image_fp in image_fps:
        image = gdal.Open(image_fp)

        # Pre-allocate an array for masked bands
        arr = np.zeros((np.shape(nlcd_mask)[0], np.shape(nlcd_mask)[1], image.RasterCount), dtype=np.uint16)

        for loc, band in enumerate(range(1, image.RasterCount+1)):   
            band_data = image.GetRasterBand(band)
            band_array = gdal_array.BandReadAsArray(band_data)
            # Generate masked band with nodata = 0
            tmp = ma.masked_array(band_array, nlcd_mask)
            arr[:, :, loc] = ma.filled(tmp,0)

        # Write image to file
        [cols, rows] = band_array.shape
        driver = gdal.GetDriverByName('GTiff')
        masked_image_fp = '%s_nlcd_masked.tif' %image_fp[:-4]
        outdata = driver.Create(masked_image_fp, rows, cols, 6, gdal.GDT_UInt16)
        outdata.SetGeoTransform(image.GetGeoTransform())##sets same geotransform as input
        outdata.SetProjection(image.GetProjection())##sets same projection as input

        for loc, band in enumerate(range(1, image.RasterCount+1)):
            outdata.GetRasterBand(band).WriteArray(arr[:, :, loc])
            outdata.GetRasterBand(band).SetNoDataValue(0)

        outdata.FlushCache() ##saves to disk!!
        outdata = None

In [5]:
nlcd_masking(image_fps, nlcd_fp, sensor)